Выполнить задачу text summarization для своего набора статей на русском языке. 
В качестве векторных представлений предложений использовать либо предобученные 
модели https://rusvectores.org/ru/models/ или собственный embending, 
полученный из SVD разложения.


In [ ]:
import re
import numpy as np
import numpy as np
import pandas as pd
import nltk
import networkx as nx
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt') # one time execution
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Word embeddings

In [ ]:
with open('/content/drive/My Drive/ML/Homework_9/model.txt', encoding='utf-8') as f:
  text = f.read()

In [ ]:
text_list = re.sub('_VERB|_NOUN|_ADV|_ADJ|_INTJ|_PROPN|_NUM', '', text).split('\n')

In [ ]:
word_embeddings = {}

for line in text_list[1:]:
  line_list = line.split(' ')
  word = line_list[0]
  vec = np.array(line_list[1:], dtype='float64')
  word_embeddings[word] = vec

## Статья

In [ ]:
with open('/content/drive/My Drive/ML/Homework_9/Article.txt', 'r') as f:
  text = f.read()

In [ ]:
text = re.sub('\n+', ' ', text)
sentences = sent_tokenize(text)

In [ ]:
sentences = [re.sub("[^а-яА-Я]+", " ", i).lower() for i in sentences]

In [ ]:
stop_words = stopwords.words('russian')

In [ ]:
sno = nltk.stem.SnowballStemmer('russian')

In [ ]:
def remove_stopwords(sen):
    sen_new = " ".join([sno.stem(i) for i in sen if sno.stem(i) not in stop_words])
    return sen_new

In [ ]:
sentences_new = [remove_stopwords(r.split()) for r in sentences]

In [ ]:
sentence_vectors = []

for i in sentences_new:
    
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((300,))
    sentence_vectors.append(v)

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [ ]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
for i in range(3):
    print(ranked_sentences[i][1])
    print('')

работа по воспитанию навыков безопасного поведения детей на улицах ни в коем случае не должна быть одноразовой акцией 

для реализации намеченной цели и поставленных задач разработали перспективный план работы по формированию навыков безопасного поведения на дороге у дошкольников приложение большое внимание уделяем работе с родителями ведь именно они каждый день не раз переходят с ребятишками дорогу и несут за них ответственность 

таким образом полученные результаты дают нам возможность предполагать что использование комплекса разнообразных мероприятий методов форм тесная работа педагогического коллектива с воспитанниками педагогами дополнительного образования инспекторами гибдд и родителями а так же систематичность и регулярность проводимой работы имеют положительное воздействие на повышение уровня безопасного поведения дошкольников на улицах города помогает нашим детям быть уверенными на дороге и избежать травматизма 



### SVD + TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences_new)

In [ ]:
svd = TruncatedSVD(n_components=8, random_state=42)
sentence_vectors = normalize(svd.fit_transform(X))

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1, 8), sentence_vectors[j].reshape(1, 8))[0,0]

In [ ]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
for i in range(3):
    print(ranked_sentences[i][1])
    print('')

активизируем работу по пропаганде правил дорожного движения и безопасного образа жизни среди родителей через разнообразные формы анкетирование родителей безопасность на дороге какой вы пешеход знаете ли вы правила дорожного движения 

приводит к этому незнание правил дорожного движения пренебрежение ими отсутствие навыков поведения на дороге а также безучастное отношение взрослых к поведению детей на дороге 

все это позволяет комплексно решать задачи обучения детей безопасному поведению в дорожной среде учитывая возрастные особенности детей уровень их психического и физического развития воспитывать дисциплинированность и сознательное выполнение правил дорожного движения культуру поведения в дорожно транспортной среде 



### TFIDF

In [ ]:
sentence_vectors = X

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1, 390), sentence_vectors[j].reshape(1, 390))[0,0]

In [ ]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
for i in range(3):
    print(ranked_sentences[i][1])
    print('')

основными целями изучения правил дорожного движения и поведения на улице являются снижение дорожно транспортного травматизма среди детей посредством повышения уровня знаний ими правил дорожного движения развитие психофизиологических качеств ребенка формирование культуры общественного поведения в процессе общения с дорогой 

активизируем работу по пропаганде правил дорожного движения и безопасного образа жизни среди родителей через разнообразные формы анкетирование родителей безопасность на дороге какой вы пешеход знаете ли вы правила дорожного движения 

все это позволяет комплексно решать задачи обучения детей безопасному поведению в дорожной среде учитывая возрастные особенности детей уровень их психического и физического развития воспитывать дисциплинированность и сознательное выполнение правил дорожного движения культуру поведения в дорожно транспортной среде 



### Текст

In [ ]:
sentences

['формирование навыков безопасности дорожного движения для дошкольников проблема дорожно транспортных происшествий дтп в россии по своим масштабам и тяжести травм имеет все признаки национальной катастрофы ',
 'особую тревогу вызывает ситуация с детским дорожно транспортным травматизмом ',
 'причиной дорожно транспортных происшествий чаще всего являются сами дети ',
 'приводит к этому незнание правил дорожного движения пренебрежение ими отсутствие навыков поведения на дороге а также безучастное отношение взрослых к поведению детей на дороге ',
 'воспитание безопасного поведения у детей одна из важнейших задач дошкольного учреждения ',
 'реб нок становится пешеходом значительно раньше чем он по своим знаниям усилиям развитию становится к этому подготовленным ',
 'с первых дней пребывания реб нка в детском саду следует так организовать его воспитание и обучение чтобы к моменту перехода из детского сада в школу он легко ориентировался в ближайшем окружении умел наблюдать и правильно оцени